In [ ]:
import pandas as pd

# Carregar os dados
df = pd.read_csv("../database/dataset gerado/dataset_limpo.csv")

# Selecionar colunas relevantes, remover nulos e normalizar
dados = (
    df[["Dim_Lojas.Bairro_Emp", "Dim_Lojas.Cidade_Emp", "Dim_Lojas.Estado_Emp"]]
    .dropna()  # remove linhas com valores nulos
    .copy()
)

# Normalizar strings
dados["Dim_Lojas.Bairro_Emp"] = dados["Dim_Lojas.Bairro_Emp"].str.strip().str.title()
dados["Dim_Lojas.Cidade_Emp"] = dados["Dim_Lojas.Cidade_Emp"].str.strip().str.title()
dados["Dim_Lojas.Estado_Emp"] = dados["Dim_Lojas.Estado_Emp"].str.strip().str.upper()

# Remover duplicatas
dados_unicos = dados.drop_duplicates()

# Ordenar por cidade, depois bairro
dados_ordenados = dados_unicos.sort_values(
    by=["Dim_Lojas.Cidade_Emp", "Dim_Lojas.Bairro_Emp"]
)

# Exportar para CSV
dados_ordenados.to_csv("../database/dataset gerado/bairros_cidades_estados.csv", index=False, encoding="utf-8")

In [ ]:
from geopy.geocoders import Nominatim
import time

# Carregar seu CSV
df = pd.read_csv("../database/dataset gerado/bairros_cidades_estados.csv")

# Inicializar o geocodificador
geolocator = Nominatim(user_agent="sua_aplicacao")

# Criar colunas para armazenar os resultados
df["Latitude"] = None
df["Longitude"] = None

# Iterar por cada Dim_Lojas.Cidade_Emp única
for idx, row in df.drop_duplicates(subset=["Dim_Lojas.Cidade_Emp", "Dim_Lojas.Estado_Emp"]).iterrows():
    query = f"{row['Dim_Lojas.Cidade_Emp']}, {row['Dim_Lojas.Estado_Emp']}, Brasil"
    try:
        loc = geolocator.geocode(query)
        if loc:
            df.loc[df["Dim_Lojas.Cidade_Emp"] == row["Dim_Lojas.Cidade_Emp"], "Latitude"] = loc.latitude
            df.loc[df["Dim_Lojas.Cidade_Emp"] == row["Dim_Lojas.Cidade_Emp"], "Longitude"] = loc.longitude
        else:
            print(f"Não encontrado: {query}")
    except Exception as e:
        print(f"Erro com {query}: {e}")
    
    time.sleep(1)  # Respeitar limites de uso do Nominatim

df.to_csv("../database/dataset gerado/com_coordenadas.csv", index=False)
